### Libraries

In [94]:
%%capture
%reset -f                        # clear all variables from the workspace
'generic imports'
import os
import pandas as pd
import sys
sys.path.append(os.path.abspath('..'))
from src import utils
import importlib
importlib.reload(utils)
from psutil import virtual_memory    
import datetime
import numpy as np                 

'machine learning imports'
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Activation, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

from sklearn import metrics
from sklearn.utils import shuffle
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split

In [95]:
print('TensorFlow version:', tf.__version__)

TensorFlow version: 2.12.0


### GPU

In [96]:
# if gpu available print name, else use cpu
if tf.test.is_gpu_available():
    print('GPU:', tf.test.gpu_device_name())
else:
    print('CPU:', tf.config.list_physical_devices('CPU'))

CPU: [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]


2023-10-30 02:05:25.610212: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-10-30 02:05:25.610371: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


### Load Data

In [97]:
# Define the augmentation method and the data directory
AUGMENTATION = 'SMOTE'
data_dir = os.path.abspath('../data')

# Load the train and test datasets
df_train, df_test = utils.load_dataset(data_directory=data_dir, 
                                       augmentation=AUGMENTATION, 
                                       ignore_columns=['mbtcp.unit_id', 
                                                       'mbtcp.trans_id'])   

Loading complete.
Training data: 1500000 rows, 85 columns. 
Test data: 381934 rows, 85 columns.


### Data Preparation

In [98]:
# Creates X_train, y_train
X_train = df_train.drop(['Attack_label', 'Attack_type'], axis=1)
y_train = df_train['Attack_type']

# Creates X_test, y_test
X_test = df_test.drop(['Attack_label', 'Attack_type'], axis=1)
y_test = df_test['Attack_type']

#### Convert categorical features to one-hot encoded features

In [99]:
# One-hot encode the training and test labels if needed
X_train_enc, X_test_enc = utils.encode_categorical(X_train, X_test)

No categorical features found. Returning original datasets.


#### Label Encoding

In [100]:
y_train_enc, y_test_enc, le = utils.encode_labels(y_train, y_test)
y_train_bin = tf.keras.utils.to_categorical(y_train_enc)
# y_test_bin = tf.keras.utils.to_categorical(y_test_enc)

Attack_type and encoded labels:

Backdoor                0
DDoS_HTTP               1
DDoS_ICMP               2
DDoS_TCP                3
DDoS_UDP                4
Fingerprinting          5
MITM                    6
Normal                  7
Password                8
Port_Scanning           9
Ransomware              10
SQL_injection           11
Uploading               12
Vulnerability_scanner   13
XSS                     14


#### Standardization of Data

In [102]:
X_train_scaled, X_test_scaled = utils.scale_data(X_train, X_test, scaler_type='standard')

          mean      std       
Train:    -0.000    0.944     
Test:     -0.010    1.871     


### Model Training

In [103]:
# Define the model
model = Sequential()
model.add(Dense(256, input_dim=X_train.shape[1], activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(128, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(32, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(len(le.classes_), activation='softmax')) 

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy']) 

# ReduceLROnPlateau callback
monitor = tf.keras.callbacks.ReduceLROnPlateau(monitor="loss",
                                               factor=0.3,
                                               mode="min",
                                               patience=10,
                                               verbose=1,
                                               min_lr=1e-8)

# Checkpoint callback                                                
checkpoint = ModelCheckpoint(f'../checkpoints/neural_net/best_model_{AUGMENTATION}.h5', 
                              monitor='loss', 
                              save_best_only=True)

In [104]:
# Shuffle training data
X_train, y_train_bin = shuffle(X_train_scaled, y_train_bin, random_state=42)

# Train the model
history = model.fit(X_train, 
                    y_train_bin, 
                    epochs=100, 
                    batch_size=512, 
                    callbacks=[monitor, checkpoint])

Epoch 1/100
2930/2930 [==============================] - 36s 12ms/step - loss: 0.5600 - accuracy: 0.7326 - lr: 0.0010
Epoch 2/100
2930/2930 [==============================] - 33s 11ms/step - loss: 0.4351 - accuracy: 0.7909 - lr: 0.0010
Epoch 3/100
2930/2930 [==============================] - 32s 11ms/step - loss: 0.3902 - accuracy: 0.8137 - lr: 0.0010
Epoch 4/100
2930/2930 [==============================] - 32s 11ms/step - loss: 0.3819 - accuracy: 0.8164 - lr: 0.0010
Epoch 5/100
2930/2930 [==============================] - 31s 11ms/step - loss: 0.3777 - accuracy: 0.8179 - lr: 0.0010
Epoch 6/100
2930/2930 [==============================] - 31s 11ms/step - loss: 0.3731 - accuracy: 0.8195 - lr: 0.0010
Epoch 7/100
2930/2930 [==============================] - 32s 11ms/step - loss: 0.3706 - accuracy: 0.8204 - lr: 0.0010
Epoch 8/100
2930/2930 [==============================] - 32s 11ms/step - loss: 0.3667 - accuracy: 0.8221 - lr: 0.0010
Epoch 9/100
2930/2930 [==============================] -

### Model Evaluation

In [105]:
# predict probabilities for test set and get the index of the highest probability
predictions = model.predict(X_test_scaled)

11936/11936 [==============================] - 17s 1ms/step


In [106]:
# get the index of the highest probability
predictions = tf.argmax(predictions, axis=1).numpy()

In [109]:
predictions = pred

In [110]:
# Calculate metrics 
accuracy = metrics.accuracy_score(y_test_enc, predictions)
precision_m = metrics.precision_score(y_test_enc, predictions, average='macro')
recall_m = metrics.recall_score(y_test_enc, predictions, average='macro')
f1_score_m = metrics.f1_score(y_test_enc, predictions, average='macro')
precision_w = metrics.precision_score(y_test_enc, predictions, average='weighted')
recall_w = metrics.recall_score(y_test_enc, predictions, average='weighted')
f1_score_w = metrics.f1_score(y_test_enc, predictions, average='weighted')

In [112]:
# create dictionary for results
results = {
    "model": "Neural Net",
    "augmentations": AUGMENTATION,
    "timestamp": datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
    "accuracy": accuracy,
    "precision_macro": precision_m,
    "recall_macro": recall_m,
    "f1_macro": f1_score_m,
    "precision_weighted": precision_w,
    "recall_weighted": recall_w,
    "f1_weighted": f1_score_w
    }

# print results
utils.print_results_table(results)

╒══════════════════════╤═════════╕
│ Metric               │ Value   │
╞══════════════════════╪═════════╡
│ Accuracy             │ 94.18%  │
├──────────────────────┼─────────┤
│ Precision (macro)    │ 76.73%  │
├──────────────────────┼─────────┤
│ Recall (macro)       │ 80.61%  │
├──────────────────────┼─────────┤
│ F1 (macro)           │ 76.48%  │
├──────────────────────┼─────────┤
│ Precision (weighted) │ 95.06%  │
├──────────────────────┼─────────┤
│ Recall (weighted)    │ 94.18%  │
├──────────────────────┼─────────┤
│ F1 (weighted)        │ 94.37%  │
╘══════════════════════╧═════════╛


#### Save Metrics Results 

In [113]:
# save results to csv   
utils.save_results_to_csv([results], '../results/metrics/neural_net.csv')

#### Confusion Matrix

In [118]:
conf_mat = metrics.confusion_matrix(y_test_enc, predictions)

# Create a dataframe from the confusion matrix
conf_mat_df = pd.DataFrame(conf_mat, 
                            index = list(le.classes_), 
                            columns = list(le.classes_))
conf_mat_df.index.name = 'Actual'
conf_mat_df.columns.name = 'Predicted'


# Save the confusion matrix
conf_mat_df.to_csv(f"../results/conf_matrix/{results['model']}_{results['augmentations']}.csv")
conf_mat_df

Predicted,Backdoor,DDoS_HTTP,DDoS_ICMP,DDoS_TCP,DDoS_UDP,Fingerprinting,MITM,Normal,Password,Port_Scanning,Ransomware,SQL_injection,Uploading,Vulnerability_scanner,XSS
Actual,,,,,,,,,,,,,,,
Backdoor,4453,0,0,6,1,169,0,0,0,76,77,0,0,0,0
DDoS_HTTP,0,7504,0,0,0,0,0,0,0,0,0,0,0,4,2120
DDoS_ICMP,0,0,13419,0,1,81,0,0,0,0,0,0,0,0,0
DDoS_TCP,0,0,0,6302,0,0,0,0,0,3707,0,0,0,0,0
DDoS_UDP,0,0,0,0,24601,0,0,0,0,0,0,0,0,0,0
Fingerprinting,0,0,0,1,0,122,0,0,0,22,1,0,0,0,0
MITM,0,0,0,0,0,0,76,0,0,0,0,0,0,0,0
Normal,10,0,0,0,0,47,0,272719,0,0,0,0,0,0,0
Password,0,0,0,0,0,0,0,0,5242,0,0,3938,928,0,0
